In [4]:
#Generate and calculate accuracy with LexRank
from lexrank import STOPWORDS, LexRank
from path import Path
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
from rouge import Rouge 


rouge = Rouge()
    
df=pd.read_csv('train.csv')

documents = []
#Training directory for lexRank. Can be obtained online http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
documents_dir = Path('bbc')
for file_path in documents_dir.files('*.txt'):
    with file_path.open(mode='rt', encoding='utf-8') as fp:
        documents.append(fp.readlines())

f_score_rouge1=0.0
f_score_rouge2=0.0
f_score_rougel=0.0
#Read top 1000 rows to compare performance on various appraches of Extractive summarization
df=df.head(1000)
for  index,row in df.iterrows():
    text= row['Content']
    manual_summary=row['Summary']
    sentences=sent_tokenize(text)
    lxr = LexRank(documents, stopwords=STOPWORDS['en'])

    # get summary with classical LexRank algorithm
    summary = lxr.get_summary(sentences, summary_size=3, threshold=.1)
    summary="".join(summary)
    #If summary or generated summary is blank make it NA
    if summary.strip()=="":
        summary="NA"
    if manual_summary.strip()=="":
        manual_summary="NA"
    scores = rouge.get_scores(summary,manual_summary)
    f_score_rouge1=f_score_rouge1+scores[0].get('rouge-1').get('f')
    f_score_rouge2=f_score_rouge2+scores[0].get('rouge-2').get('f')
    f_score_rougel=f_score_rougel+scores[0].get('rouge-l').get('f')
print (f_score_rouge1/1000)
print (f_score_rouge2/1000)
print (f_score_rougel/1000)


0.26916337154164033
0.07930638144386636
0.22595358950279854


In [5]:
#Calculate score using gensim
from gensim.summarization.summarizer import summarize
f_score_rouge1=0.0
f_score_rouge2=0.0
f_score_rougel=0.0
#Read top 1000 rows to compare performance on various appraches of Extractive summarization
df=df.head(1000)
for  index,row in df.iterrows():
    text= row['Content']
    manual_summary=row['Summary']
    summary=summarize(text)
    summary="".join(summary)
    #If summary or generated summary is blank make it NA
    if summary.strip()=="":
        summary="NA"
    if manual_summary.strip()=="":
        manual_summary="NA"
    scores = rouge.get_scores(summary,manual_summary)
    f_score_rouge1=f_score_rouge1+scores[0].get('rouge-1').get('f')
    f_score_rouge2=f_score_rouge2+scores[0].get('rouge-2').get('f')
    f_score_rougel=f_score_rougel+scores[0].get('rouge-l').get('f')
print (f_score_rouge1/1000)
print (f_score_rouge2/1000)
print (f_score_rougel/1000)


0.23072314908431696
0.08407873447099477
0.22219688866672083


In [6]:
#Custom implementation for extractive summarization
import sys
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import numpy as np


#Preprocess function. lower case and no stop words
def preprocess(sentence):
    return [word.lower() for word in word_tokenize(sentence) if (word not in stopwords and word.isalpha())]

def text_rank_sent(graph,node_weights,d=.85,iterations=20):
    weight_sum = np.sum(graph,axis=0)
    print (weight_sum)
    while iterations >0:
        for i in range(len(node_weights)):
            temp = 0.0
            for j in range(len(node_weights)):
                temp += graph[i,j]*node_weights[j]/weight_sum[j]
                print (graph[i,j],node_weights[j],weight_sum[j],graph[i,j]*node_weights[j]/weight_sum[j])
            print ("temp",i, temp)
            node_weights[i] = 1-d+(d*temp)
        iterations-=1

def pageRankSummary(text):
    #split text into sentences using nltk 
    sentences = sent_tokenize(text)
    #Preprocess all sentences
    processed_sentence = list(map(preprocess,sentences))
    #Total number of nodes(sentences)
    nodes_count = len(list(processed_sentence))

    #create an empty matrix of N x N. N is number of sentences
    graph = np.zeros((nodes_count,nodes_count)) 
    for i in range(nodes_count):
        try:
            for j in range(i+1,nodes_count):
                #TextRank computation. Common words in sentences i & j divide by total words in i &j
                graph[i,j] = float(len(set(processed_sentence[i])&set(processed_sentence[j])))/(len(processed_sentence[i])+len(processed_sentence[j]))
                graph[j,i] = graph[i,j]
        except:
            continue
    #Method to calculate Page Rank (PR) score for each and every sentence.
    #PR of sentence x is simply sum of score of all sentences pointing to x.
    #Final score is multiplied by a damping factor, generally 0.85
    node_weights = np.ones(nodes_count)
    #Print Top N sentences

    top_n = 3
    top_index = [i for i,j in sorted(enumerate(node_weights), key=lambda x: x[1],reverse=True)[:top_n]]

    summary = [sentences[i] for i in top_index]
    return summary

f_score_rouge1=0.0
f_score_rouge2=0.0
f_score_rougel=0.0
#Read top 1000 rows to compare performance on various appraches of Extractive summarization
df=df.head(1000)
#Load english stop words
stopwords = set(stopwords.words('english'))
for  index,row in df.iterrows():
    text= row['Content']
    manual_summary=row['Summary']
    summary=pageRankSummary(text)
    #print(summary)
    summary="".join(summary)
    #If summary or generated summary is blank make it NA
    if summary.strip()=="":
        summary="NA"
    if manual_summary.strip()=="":
        manual_summary="NA"
    scores = rouge.get_scores(summary,manual_summary)
    f_score_rouge1=f_score_rouge1+scores[0].get('rouge-1').get('f')
    f_score_rouge2=f_score_rouge2+scores[0].get('rouge-2').get('f')
    f_score_rougel=f_score_rougel+scores[0].get('rouge-l').get('f')
print (f_score_rouge1/1000)
print (f_score_rouge2/1000)
print (f_score_rougel/1000)



0.2872083426024757
0.10306745872514284
0.2449744538786308


With our experiments on 3 different approaches we found our custom pagerank based extractive approach turning out to be the best. So we are going to PageRank based extractive summarization for test data.

In [7]:
df_test=pd.read_csv('test.csv')
dfObj = pd.DataFrame()
for  index,row in df_test.iterrows():
    text= row['Content']
    summary=pageRankSummary(text)
    #print(summary)
    summary="".join(summary)
    #If summary or generated summary is blank make it NA
    if summary.strip()=="":
        summary="NA"
    if manual_summary.strip()=="":
        manual_summary="NA"
    tempdict={}    
    tempdict['ID']=row['ID']
    tempdict['Summary']=summary
    tempdf  = pd.DataFrame([tempdict], columns=tempdict.keys())
    dfObj = pd.concat([dfObj, tempdf])
dfObj.to_csv('sumbmission.csv', mode='a', header=True)

